In [60]:
import json
import uuid
import numpy as np
import pandas as pd
import requests as rq

from pandas.io.json import json_normalize

## Обычно мобильное API позволяет вызывать методы из стандартного API как Legacy-методы.

Документация к API Яндекс.Маркет: https://yandex.ru/dev/market/content-data/doc/dg-v2/concepts/versions-docpage/

---

# Получение дерева категорий Яндекс.Маркет

## Путь 1 - Скачать официальный файл

https://market.yandex.ru/partners/education/market-basis/tovarnye-predlozheniya-na-markete

https://yandex.ru/support/partnermarket/guides/classification.html

In [12]:
! wget http://download.cdn.yandex.net/market/market_categories.xls

--2020-05-03 14:47:58--  http://download.cdn.yandex.net/market/market_categories.xls
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.242, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskmar09.cdn.yandex.net/download.cdn.yandex.net/market/market_categories.xls [following]
--2020-05-03 14:47:58--  http://cache-mskmar09.cdn.yandex.net/download.cdn.yandex.net/market/market_categories.xls
Resolving cache-mskmar09.cdn.yandex.net (cache-mskmar09.cdn.yandex.net)... 5.45.222.33
Connecting to cache-mskmar09.cdn.yandex.net (cache-mskmar09.cdn.yandex.net)|5.45.222.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 744448 (727K) [application/vnd.ms-excel]
Saving to: ‘market_categories.xls’

market_categories.x 100%[===================>] 727.00K   259KB/s    in 2.8s    

2020-05-03 14:48:01 (259 KB/

In [38]:
df_categories = pd.read_excel('market_categories.xls', sep='/')
df_categories.head(10)

,Все товары
0,Все товары/Авто
1,Все товары/Авто/Автомобильные инструменты
2,Все товары/Авто/Автомобильные инструменты/Авто...
3,Все товары/Авто/Автомобильные инструменты/Вспо...
4,Все товары/Авто/Автомобильные инструменты/Вспо...
5,Все товары/Авто/Автомобильные инструменты/Диаг...
6,Все товары/Авто/Автомобильные инструменты/Диаг...
7,Все товары/Авто/Автомобильные инструменты/Диаг...
8,Все товары/Авто/Автомобильные инструменты/Диаг...
9,Все товары/Авто/Автомобильные инструменты/Диаг...


In [98]:
len(df_categories)

3774

In [39]:
for index, row in df_categories.iterrows():
    tmp_list = row['Все товары'].split('/')
    for n, item in enumerate(tmp_list):
        df_categories.at[index, str(n)] = item
        
df_categories.head(10)

,Все товары,0,1,2,3,4,5,6
0,Все товары/Авто,Все товары,Авто,NaN,NaN,NaN,NaN,NaN
1,Все товары/Авто/Автомобильные инструменты,Все товары,Авто,Автомобильные инструменты,NaN,NaN,NaN,NaN
2,Все товары/Авто/Автомобильные инструменты/Авто...,Все товары,Авто,Автомобильные инструменты,Автомобильные компрессоры,NaN,NaN,NaN
3,Все товары/Авто/Автомобильные инструменты/Вспо...,Все товары,Авто,Автомобильные инструменты,Вспомогательный инструмент,Инструмент для ГРМ,NaN,NaN
4,Все товары/Авто/Автомобильные инструменты/Вспо...,Все товары,Авто,Автомобильные инструменты,Вспомогательный инструмент,Фиксаторы и зажимы,NaN,NaN
5,Все товары/Авто/Автомобильные инструменты/Диаг...,Все товары,Авто,Автомобильные инструменты,Диагностическое и измерительное оборудование,Видеоэндоскопы,NaN,NaN
6,Все товары/Авто/Автомобильные инструменты/Диаг...,Все товары,Авто,Автомобильные инструменты,Диагностическое и измерительное оборудование,Диагностическое оборудование,NaN,NaN
7,Все товары/Авто/Автомобильные инструменты/Диаг...,Все товары,Авто,Автомобильные инструменты,Диагностическое и измерительное оборудование,Компрессометры,NaN,NaN
8,Все товары/Авто/Автомобильные инструменты/Диаг...,Все товары,Авто,Автомобильные инструменты,Диагностическое и измерительное оборудование,Манометры,NaN,NaN
9,Все товары/Авто/Автомобильные инструменты/Диаг...,Все товары,Авто,Автомобильные инструменты,Диагностическое и измерительное оборудование,Стетоскопы и стробоскопы,NaN,NaN


---

## Путь 2 - Legacy-метод в мобильном API Яндекс.Маркет

https://yandex.ru/dev/market/content-data/doc/dg-v2/reference/category-controller-v2-get-children-categories-docpage/

In [52]:
tmp_uuid = uuid.uuid4().hex

headers = {
    'Accept': '*/*',
    'X-Platform': 'IOS',
    'Api-Platform': 'IOS',
    'Accept-Language': 'en-us',
    'Connection': 'keep-alive',
    'X-App-Version': '600.13.1',
    'X-Device-Type': 'SMARTPHONE',
    'Host': 'mobile.market.yandex.net',
    'Accept-Encoding': 'gzip, deflate, br',
}

response = rq.get(
    'https://mobile.market.yandex.net/market/white/v2.1.5/categories',
    params={'uuid':tmp_uuid, 'count':30},
    headers=headers
)
response.status_code

200

In [54]:
json_normalize(response.json()['categories'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,fullName,type,link,childCount,advertisingModel,viewType
0,17392030,Упаковочные материалы для Беру,Упаковочные материалы для Беру,GENERAL,https://market.yandex.ru/catalog--upakovochnye...,0,CPC,LIST
1,90402,Авто,Товары для авто- и мототехники,GENERAL,https://market.yandex.ru/catalog--tovary-dlia-...,12,CPC,LIST
2,90509,Товары для красоты,Товары для красоты,GENERAL,https://market.yandex.ru/catalog--tovary-dlia-...,3,CPC,LIST
3,90666,Товары для дома,Товары для дома,GENERAL,https://market.yandex.ru/catalog--tovary-dlia-...,9,CPC,LIST
4,90719,"Дача, сад и огород","Дача, сад и огород",GENERAL,https://market.yandex.ru/catalog--dacha-sad-i-...,13,CPC,LIST
5,90764,Детские товары,Детские товары,GENERAL,https://market.yandex.ru/catalog--detskie-tova...,6,CPC,LIST
6,90801,Досуг и развлечения,Досуг и развлечения,GENERAL,https://market.yandex.ru/catalog--dosug-i-razv...,18,CPC,LIST
7,90813,Товары для животных,Товары для животных,GENERAL,https://market.yandex.ru/catalog--tovary-dlia-...,31,CPC,LIST
8,91009,Компьютерная техника,Компьютерная техника,GENERAL,https://market.yandex.ru/catalog--kompiuternai...,12,CPC,LIST
9,91307,Продукты,"Продукты, напитки",GENERAL,https://market.yandex.ru/catalog--produkty-nap...,16,CPC,LIST


In [68]:
df = pd.DataFrame(columns=['id', 'name', 'fullName', 'link', 'level', 'parent', 'childCount'])

for index, row in json_normalize(response.json()['categories']).iterrows():
    df = df.append(
        pd.Series(
            {
                'id':row.id,
                'name':row['name'],
                'fullName':row.fullName,
                'link':row.link,
                'level':0,
                'childCount':row.childCount
            }
        ), ignore_index=True
    )
    
df

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,fullName,link,level,parent,childCount
0,17392030,Упаковочные материалы для Беру,Упаковочные материалы для Беру,https://market.yandex.ru/catalog--upakovochnye...,0,NaN,0
1,90402,Авто,Товары для авто- и мототехники,https://market.yandex.ru/catalog--tovary-dlia-...,0,NaN,12
2,90509,Товары для красоты,Товары для красоты,https://market.yandex.ru/catalog--tovary-dlia-...,0,NaN,3
3,90666,Товары для дома,Товары для дома,https://market.yandex.ru/catalog--tovary-dlia-...,0,NaN,9
4,90719,"Дача, сад и огород","Дача, сад и огород",https://market.yandex.ru/catalog--dacha-sad-i-...,0,NaN,13
5,90764,Детские товары,Детские товары,https://market.yandex.ru/catalog--detskie-tova...,0,NaN,6
6,90801,Досуг и развлечения,Досуг и развлечения,https://market.yandex.ru/catalog--dosug-i-razv...,0,NaN,18
7,90813,Товары для животных,Товары для животных,https://market.yandex.ru/catalog--tovary-dlia-...,0,NaN,31
8,91009,Компьютерная техника,Компьютерная техника,https://market.yandex.ru/catalog--kompiuternai...,0,NaN,12
9,91307,Продукты,"Продукты, напитки",https://market.yandex.ru/catalog--produkty-nap...,0,NaN,16


In [71]:
for index, row in df[(df.level == 0) & (df.childCount > 0)].iterrows():
    tmp_response = rq.get(
        f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
        params={'uuid':tmp_uuid, 'count':30},
        headers=headers
    )
    
    for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
        df = df.append(
            pd.Series(
                {
                    'id':row2.id,
                    'name':row2['name'],
                    'fullName':row2.fullName,
                    'link':row2.link,
                    'level':1,
                    'childCount':row2.childCount,
                    'parent':row.id
                }
            ), ignore_index=True
        )
    
    cur_page = 1
    max_page = tmp_response.json()['context']['page']['total']
    while cur_page < max_page:
        cur_page += 1
        tmp_response = rq.get(
            f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
            params={'uuid':tmp_uuid, 'count':30, 'page':cur_page},
            headers=headers
        )
        
        for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
            df = df.append(
                pd.Series(
                    {
                        'id':row2.id,
                        'name':row2['name'],
                        'fullName':row2.fullName,
                        'link':row2.link,
                        'level':1,
                        'childCount':row2.childCount,
                        'parent':row.id
                    }
                ), ignore_index=True
            )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [77]:
df[df.level == 1].childCount.sum()

1196

In [76]:
for index, row in df[(df.level == 1) & (df.childCount > 0)].iterrows():
    tmp_response = rq.get(
        f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
        params={'uuid':tmp_uuid, 'count':30},
        headers=headers
    )
    
    for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
        df = df.append(
            pd.Series(
                {
                    'id':row2.id,
                    'name':row2['name'],
                    'fullName':row2.fullName,
                    'link':row2.link,
                    'level':2,
                    'childCount':row2.childCount,
                    'parent':row.id
                }
            ), ignore_index=True
        )
    
    cur_page = 1
    max_page = tmp_response.json()['context']['page']['total']
    while cur_page < max_page:
        cur_page += 1
        tmp_response = rq.get(
            f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
            params={'uuid':tmp_uuid, 'count':30, 'page':cur_page},
            headers=headers
        )
        
        for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
            df = df.append(
                pd.Series(
                    {
                        'id':row2.id,
                        'name':row2['name'],
                        'fullName':row2.fullName,
                        'link':row2.link,
                        'level':2,
                        'childCount':row2.childCount,
                        'parent':row.id
                    }
                ), ignore_index=True
            )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [78]:
len(df[df.level == 2])

1195

In [79]:
df[df.level == 2].childCount.sum()

1797

In [80]:
df[df.level == 2].head(10)

,id,name,fullName,link,level,parent,childCount
222,90404,Автомагнитолы,Автомагнитолы,https://market.yandex.ru/catalog--avtomagnitol...,2,90403,0
223,90407,Антенны,Автомобильные антенны,https://market.yandex.ru/catalog--avtomobilnye...,2,90403,0
224,90409,Аксессуары,Аксессуары для автомобильной аудиотехники,https://market.yandex.ru/catalog--aksessuary-d...,2,90403,0
225,90408,Усилители,Автомобильные усилители,https://market.yandex.ru/catalog--avtomobilnye...,2,90403,0
226,90417,Автомобильные телевизоры,Автомобильные телевизоры и мониторы,https://market.yandex.ru/catalog--avtomobilnye...,2,90403,0
227,7717599,Переходные рамки,Переходные рамки для автомагнитол,https://market.yandex.ru/catalog--perekhodnye-...,2,90403,0
228,289012,"Акустические полки, короба и подиумы","Акустические полки, короба и подиумы для автом...",https://market.yandex.ru/catalog--akusticheski...,2,90403,0
229,6122737,FM-трансмиттеры,Автомобильные FM-трансмиттеры,https://market.yandex.ru/catalog--avtomobilnye...,2,90403,0
230,308016,Автоакустика,Автомобильная акустика,https://market.yandex.ru/catalog--avtomobilnai...,2,90403,0
231,9335976,Шумоизоляция,Шумоизоляция для автомобилей,https://market.yandex.ru/catalog--shumoizoliat...,2,90403,0


In [81]:
for index, row in df[(df.level == 2) & (df.childCount > 0)].iterrows():
    tmp_response = rq.get(
        f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
        params={'uuid':tmp_uuid, 'count':30},
        headers=headers
    )
    
    for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
        df = df.append(
            pd.Series(
                {
                    'id':row2.id,
                    'name':row2['name'],
                    'fullName':row2.fullName,
                    'link':row2.link,
                    'level':3,
                    'childCount':row2.childCount,
                    'parent':row.id
                }
            ), ignore_index=True
        )
    
    cur_page = 1
    max_page = tmp_response.json()['context']['page']['total']
    while cur_page < max_page:
        cur_page += 1
        tmp_response = rq.get(
            f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
            params={'uuid':tmp_uuid, 'count':30, 'page':cur_page},
            headers=headers
        )
        
        for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
            df = df.append(
                pd.Series(
                    {
                        'id':row2.id,
                        'name':row2['name'],
                        'fullName':row2.fullName,
                        'link':row2.link,
                        'level':3,
                        'childCount':row2.childCount,
                        'parent':row.id
                    }
                ), ignore_index=True
            )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [82]:
len(df[df.level == 3])

1796

In [83]:
df[df.level == 3].childCount.sum()

464

In [84]:
df[df.level == 3].head(10)

,id,name,fullName,link,level,parent,childCount
1417,14369216,Очистители для автостекол,Очистители для автостекол,https://market.yandex.ru/catalog--ochistiteli-...,3,14369211,0
1418,14369215,Антизапотеватели,Антизапотеватели для стекол автомобилей,https://market.yandex.ru/catalog--antizapoteva...,3,14369211,0
1419,14369214,Антидождь,Антидождь для автостекол,https://market.yandex.ru/catalog--antidozhd-dl...,3,14369211,0
1420,14369311,Полироли для фар и автостекол,Полироли для фар и автостекол,https://market.yandex.ru/catalog--poliroli-dli...,3,14369211,0
1421,14201356,Жидкость для стеклоомывателя,Жидкость для стеклоомывателя,https://market.yandex.ru/catalog--zhidkost-dli...,3,14369211,0
1422,14369423,Очистители и полироли для шин и дисков,Очистители и полироли для шин и дисков автомоб...,https://market.yandex.ru/catalog--ochistiteli-...,3,14369377,0
1423,14369455,Наборы и герметики для ремонта шин,Наборы и герметики для ремонта шин автомобилей,https://market.yandex.ru/catalog--nabory-i-ger...,3,14369377,0
1424,90433,Шпатлевки,Шпатлевки для кузовного ремонта автомобилей,https://market.yandex.ru/catalog--shpatlevki-d...,3,14369611,0
1425,90432,Краски и лаки,Краски и лаки для кузовного ремонта автомобилей,https://market.yandex.ru/catalog--kraski-i-lak...,3,14369611,2
1426,90434,Грунты,Грунты для кузовного ремонта автомобилей,https://market.yandex.ru/catalog--grunty-dlia-...,3,14369611,0


In [85]:
for index, row in df[(df.level == 3) & (df.childCount > 0)].iterrows():
    tmp_response = rq.get(
        f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
        params={'uuid':tmp_uuid, 'count':30},
        headers=headers
    )
    
    for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
        df = df.append(
            pd.Series(
                {
                    'id':row2.id,
                    'name':row2['name'],
                    'fullName':row2.fullName,
                    'link':row2.link,
                    'level':4,
                    'childCount':row2.childCount,
                    'parent':row.id
                }
            ), ignore_index=True
        )
    
    cur_page = 1
    max_page = tmp_response.json()['context']['page']['total']
    while cur_page < max_page:
        cur_page += 1
        tmp_response = rq.get(
            f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
            params={'uuid':tmp_uuid, 'count':30, 'page':cur_page},
            headers=headers
        )
        
        for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
            df = df.append(
                pd.Series(
                    {
                        'id':row2.id,
                        'name':row2['name'],
                        'fullName':row2.fullName,
                        'link':row2.link,
                        'level':4,
                        'childCount':row2.childCount,
                        'parent':row.id
                    }
                ), ignore_index=True
            )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [88]:
len(df[df.level == 4])

464

In [89]:
df[df.level == 4].childCount.sum()

23

In [90]:
df[df.level == 4].head(10)

,id,name,fullName,link,level,parent,childCount
3213,14369617,Краски,Краски для кузовного ремонта автомобилей,https://market.yandex.ru/catalog--kraski-dlia-...,4,90432,0
3214,16729315,Лаки,Лаки для кузовного ремонта автомобилей,https://market.yandex.ru/catalog--laki-dlia-ku...,4,90432,0
3215,13885651,Комплектующие к ксенону,Комплектующие к ксенону,https://market.yandex.ru/catalog--komplektuius...,4,17692691,3
3216,599239,Комплекты ксенона,Комплекты ксенона,https://market.yandex.ru/catalog--komplekty-ks...,4,17692691,0
3217,16808461,Накладки на пороги,Автомобильные накладки на пороги и бамперы,https://market.yandex.ru/catalog--avtomobilnye...,4,13208052,0
3218,16808666,Защита порогов и пороги-площадки,Защита порогов и пороги-площадки,https://market.yandex.ru/catalog--zashchita-po...,4,13208052,0
3219,14994924,Краски,Акриловые и гелевые краски для дизайна ногтей,https://market.yandex.ru/catalog--akrilovye-i-...,4,13239017,0
3220,14994948,"Наклейки, трафареты, фольга","Наклейки, трафареты, фольга для дизайна ногтей",https://market.yandex.ru/catalog--nakleiki-tra...,4,13239017,0
3221,16076565,Инструменты,Инструменты и принадлежности для дизайна ногтей,https://market.yandex.ru/catalog--instrumenty-...,4,13239017,0
3222,14990255,Стемпинг,Товары для стемпинга,https://market.yandex.ru/catalog--tovary-dlia-...,4,13239017,0


In [91]:
for index, row in df[(df.level == 4) & (df.childCount > 0)].iterrows():
    tmp_response = rq.get(
        f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
        params={'uuid':tmp_uuid, 'count':30},
        headers=headers
    )
    
    for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
        df = df.append(
            pd.Series(
                {
                    'id':row2.id,
                    'name':row2['name'],
                    'fullName':row2.fullName,
                    'link':row2.link,
                    'level':5,
                    'childCount':row2.childCount,
                    'parent':row.id
                }
            ), ignore_index=True
        )
    
    cur_page = 1
    max_page = tmp_response.json()['context']['page']['total']
    while cur_page < max_page:
        cur_page += 1
        tmp_response = rq.get(
            f'https://mobile.market.yandex.net/market/white/v2.1.5/categories/{row.id}/children',
            params={'uuid':tmp_uuid, 'count':30, 'page':cur_page},
            headers=headers
        )
        
        for index2, row2 in json_normalize(tmp_response.json()['categories']).iterrows():
            df = df.append(
                pd.Series(
                    {
                        'id':row2.id,
                        'name':row2['name'],
                        'fullName':row2.fullName,
                        'link':row2.link,
                        'level':5,
                        'childCount':row2.childCount,
                        'parent':row.id
                    }
                ), ignore_index=True
            )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [92]:
len(df[df.level == 5])

23

In [93]:
df[df.level == 5].childCount.sum()

0

In [94]:
df[df.level == 5].head(10)

,id,name,fullName,link,level,parent,childCount
3677,17613580,Биксеноновые линзы,Биксеноновые линзы,https://market.yandex.ru/catalog--biksenonovye...,5,13885651,0
3678,17613704,Прочие комплектующие,Прочие комплектующие к ксенону,https://market.yandex.ru/catalog--prochie-komp...,5,13885651,0
3679,17613548,Блоки розжига,Блоки розжига,https://market.yandex.ru/catalog--bloki-rozzhi...,5,13885651,0
3680,15220918,Кладочные смеси и клеи специального назначения,Кладочные смеси и клеи специального назначения,https://market.yandex.ru/catalog--kladochnye-s...,5,15157758,0
3681,15059051,Клей для плитки и камня,Клей для плитки и камня,https://market.yandex.ru/catalog--klei-dlia-pl...,5,15157758,0
3682,91717,"Щебень, гравий, керамзит","Щебень, гравий, керамзит",https://market.yandex.ru/catalog--shcheben-gra...,5,15157864,0
3683,15132435,"Песок, глина","Песок, глина",https://market.yandex.ru/catalog--pesok-glina/...,5,15157864,0
3684,4967967,Цемент,Цемент,https://market.yandex.ru/catalog--tsement/4967...,5,15157834,0
3685,15142683,Пескобетон,Пескобетон,https://market.yandex.ru/catalog--peskobeton/1...,5,15157834,0
3686,15096278,"Известь, гипс, мел","Вапка, гипс, мел",https://market.yandex.ru/catalog--vapka-gips-m...,5,15157834,0


In [95]:
df

,id,name,fullName,link,level,parent,childCount
0,17392030,Упаковочные материалы для Беру,Упаковочные материалы для Беру,https://market.yandex.ru/catalog--upakovochnye...,0,NaN,0
1,90402,Авто,Товары для авто- и мототехники,https://market.yandex.ru/catalog--tovary-dlia-...,0,NaN,12
2,90509,Товары для красоты,Товары для красоты,https://market.yandex.ru/catalog--tovary-dlia-...,0,NaN,3
3,90666,Товары для дома,Товары для дома,https://market.yandex.ru/catalog--tovary-dlia-...,0,NaN,9
4,90719,"Дача, сад и огород","Дача, сад и огород",https://market.yandex.ru/catalog--dacha-sad-i-...,0,NaN,13
...,...,...,...,...,...,...,...
3695,15273614,Вспененный полиэтилен,Вспененный полиэтилен,https://market.yandex.ru/catalog--vspenennyi-p...,5,15273517,0
3696,15273601,Трубная изоляция,Теплоизоляция для труб,https://market.yandex.ru/catalog--teploizoliat...,5,15273517,0
3697,15273597,Межвенцовый утеплитель,Межвенцовый утеплитель,https://market.yandex.ru/catalog--mezhventsovy...,5,15273517,0
3698,15273598,"Пенополистирол, Пенопласт, PIR плиты","Пенополистирол, Пенопласт, PIR плиты",https://market.yandex.ru/catalog--penopolistir...,5,15273517,0


In [99]:
df.to_excel('03052020_mgcrp_yandexMarketCategories.xlsx', index=False)

---

# Получение лучших товаров в категории с ограничением по максимальной цене

In [113]:
tmp_headers = {
    "Host" : "ipa.touch.market.yandex.ru",
    "X-Test-Id" : "",
    "Connection" : "keep-alive",
    "Accept" : "*/*",
    "Accept-Language" : "en-us",
    "Accept-Encoding" : "gzip, deflate, br",
    "Content-Type" : "application/json",
    "Api-Platform" : "IOS",
#     "X-YANDEXUID" : "8480975571588514131",
    "User-Agent" : "WhiteMarket/600.19.1 (ru.yandex.ymarket; build:1172; iOS 13.3.1; iPhone)",
#     "X-User-Authorization" : "OAuth AgAAAAA9mGxuAAGvFxQtfruNv05VpKEXPoiDkl8",
    "X-Platform" : "IOS",
    "Content-Length" : "314",
    "X-App-Version" : "600.19.1",
#     "Cookie" : "yandexuid=8480975571588514131",
    "X-Device-Type" : "SMARTPHONE"
}
    
payload = json.dumps(
    {
        "params": [
            {
                "filters": {
                    "allow-collapsing": "1",
                    "glfilter": [],
                    "grhow": "shop",
                    "onstock": "1",
                    "pricefrom": 5000,
                    "priceto": 15000,
                    "show-cutprice": "0"
                },
                "hid": 91491,
                "local-offers-first": 0,
                "numdoc": 15,
                "page": 1,
                "pp": "SEARCH",
                "rearr-factors": "search_offline_offers=1",
                "show-shops": "all",
                "show-vendors": "all",
                "text": "",
                "use-default-offers": "1"
            }
        ]
    }
)

tmp_response = rq.post(
    "https://ipa.touch.market.yandex.ru/api/v2",
    params={
        "name" : "resolveSearch",
        "rearr_factors" : "market_disable_parametric_search_for_white_except_parametric_specification=0",
        "sections" : "MEDICINE",
        "uuid" : tmp_uuid
    },
    headers=tmp_headers,
    data=payload
)

tmp_response.status_code

200

In [119]:
with open('data.json', 'w') as f:
    json.dump(tmp_response.json(), f)

In [121]:
df_goods = pd.DataFrame(columns=[
    'id', 'raw_title', 'description', 'offersCount', 'reviewsCount',
    'min_price', 'max_price', 'avg_price', 'rating', 'ratingCount',
])
for i in tmp_response.json()['collections']['product']:
    df_goods = df_goods.append(
        pd.Series(
            {
                'id'           : i['id'],
                'description'  : i['description'],
                'offersCount'  : i['offersCount'],
                'reviewsCount' : i['reviewsCount'],
                'min_price'    : i['prices']['min'],
                'max_price'    : i['prices']['max'],
                'avg_price'    : i['prices']['avg'],
                'rating'       : i['rating'],
                'ratingCount'  : i['ratingCount'],
                'raw_title'    : i['titles']['raw']
            }
        ), ignore_index=True
    )
    
df_goods

,id,description,offersCount,reviewsCount,min_price,max_price,avg_price,rating,ratingCount,raw_title
0,417636374,"тип: смартфон, android, разрешение экрана: 234...",56,168,11450,14868,12990,4.5,1024,Смартфон Honor 10i 128GB
1,419237008,"тип: смартфон, android, разрешение экрана: 231...",55,144,10790,14890,12490,4.5,822,Смартфон HUAWEI P30 lite
2,419572807,"тип: смартфон, android, разрешение экрана: 152...",88,168,7190,9490,8440,4.5,1192,Смартфон Samsung Galaxy A10
3,432102063,"тип: смартфон, линейка: Galaxy A, android, раз...",61,231,12608,14790,14790,4.5,1040,Смартфон Samsung Galaxy A40 64GB
4,562099108,"тип: смартфон, линейка: Galaxy A, разрешение э...",88,64,9490,14990,10990,4.5,473,Смартфон Samsung Galaxy A30s 32GB
5,573324027,"тип: смартфон, линейка: Redmi Note, диагональ ...",13,201,14669,14990,14990,4.5,1022,Смартфон Xiaomi Redmi Note 8 Pro 6/64GB
6,591295002,"тип: смартфон, линейка: Redmi, диагональ экран...",105,147,9537,12804,10600,4.5,760,Смартфон Xiaomi Redmi 8 4/64GB
7,630030594,"тип: смартфон, разрешение экрана: 2340×1080, 4...",98,195,11790,14990,13650,4.5,907,Смартфон Xiaomi Redmi Note 8T 4/64GB
8,646830005,"тип: смартфон, разрешение экрана: 1560x720, 4G...",22,50,10988,11990,11990,4.5,149,Смартфон ZTE Blade 20 Smart
9,650874000,"тип: смартфон, линейка: Galaxy A, диагональ эк...",13,117,14880,14990,14990,4.5,499,Смартфон Samsung Galaxy A51 64GB


In [135]:
def getTopInCategory(n, category_id, max_price, min_price=0, page=1):
    """
    Возвращает `n` первых товаров в категории `category_id` заданым
    ограничением по цене (сортировка по популярности на Яндекс.Маркет)
    
    Параметры:
        n           - int - количество товаров
        category_id - int - ID категории товара
        max_price   - int - максимальная цена товара
        min_price   - int - минимальная цена товара
        page        - int - номер страницы в выдаче (пока что оставил как технический параметр)
    """
    
    _headers = {
        "Host" : "ipa.touch.market.yandex.ru",
        "X-Test-Id" : "",
        "Connection" : "keep-alive",
        "Accept" : "*/*",
        "Accept-Language" : "en-us",
        "Accept-Encoding" : "gzip, deflate, br",
        "Content-Type" : "application/json",
        "Api-Platform" : "IOS",
        "User-Agent" : "WhiteMarket/600.19.1 (ru.yandex.ymarket; build:1172; iOS 13.3.1; iPhone)",
        "X-Platform" : "IOS",
        "Content-Length" : "314",
        "X-App-Version" : "600.19.1",
        "X-Device-Type" : "SMARTPHONE"
    }
    
    _uuid = uuid.uuid4().hex
    
    _payload = json.dumps(
        {
            "params": [
                {
                    "filters": {
                        "allow-collapsing": "1",
                        "glfilter": [],
                        "grhow": "shop",
                        "onstock": "1",
                        "pricefrom": min_price,
                        "priceto": max_price,
                        "show-cutprice": "0"
                    },
                    "hid": category_id,
                    "local-offers-first": 0,
                    "numdoc": n,
                    "page": page,
                    "pp": "SEARCH",
                    "rearr-factors": "search_offline_offers=1",
                    "show-shops": "all",
                    "show-vendors": "all",
                    "text": "",
                    "use-default-offers": "1"
                }
            ]
        }
    )
    
    _response = rq.post(
        "https://ipa.touch.market.yandex.ru/api/v2",
        params={
            "name" : "resolveSearch",
            "rearr_factors" : "market_disable_parametric_search_for_white_except_parametric_specification=0",
            "sections" : "MEDICINE",
            "uuid" : tmp_uuid
        },
        headers=_headers,
        data=_payload
    )
    
    _goods = pd.DataFrame(columns=[
        'id', 'raw_title', 'description',
        'offersCount', 'reviewsCount', 'min_price',
        'max_price', 'avg_price', 'rating', 'ratingCount'
    ])

    for item in _response.json()['collections']['product']:
        _goods = _goods.append(
            pd.Series(
                {
                    'id'           : item['id']            if 'id' in item.keys() else pd.np.nan,
                    'description'  : item['description']   if 'description' in item.keys() else pd.np.nan,
                    'offersCount'  : item['offersCount']   if 'offersCount' in item.keys() else pd.np.nan,
                    'reviewsCount' : item['reviewsCount']  if 'reviewsCount' in item.keys() else pd.np.nan,
                    'min_price'    : item['prices']['min'] if 'prices' in item.keys() else pd.np.nan,
                    'max_price'    : item['prices']['max'] if 'prices' in item.keys() else pd.np.nan,
                    'avg_price'    : item['prices']['avg'] if 'prices' in item.keys() else pd.np.nan,
                    'rating'       : item['rating']        if 'rating' in item.keys() else pd.np.nan,
                    'ratingCount'  : item['ratingCount']   if 'ratingCount' in item.keys() else pd.np.nan,
                    'raw_title'    : item['titles']['raw'] if 'titles' in item.keys() else pd.np.nan
                }
            ), ignore_index=True
        )
    
    return _goods

In [136]:
getTopInCategory(20, 15959385, 4000)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:73: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,id,raw_title,description,offersCount,reviewsCount,min_price,max_price,avg_price,rating,ratingCount
0,245430059,Набор игрушек для кошек KONG Черепашка с тубом...,"игрушка, материал: текстиль, вид животного: ко...",4,1,302,450,331,0,0
1,250195270,Игрушка для собак KONG Extreme XL (UXLE),"игрушка, материал: резина, вид животного: собаки",7,6,738,1755,1312,5,23
2,250195967,Косточка для собак Petstages Orka mini (221REX),"косточка, материал: резина, вид животного: собаки",10,6,247,750,276,4.5,11
3,250196107,Игрушка для собак KONG Classic M (T2),"игрушка, материал: резина, вид животного: собаки",9,6,662,1630,900,5,26
4,250197064,Мячик для собак KONG под лакомства (KB2E),"мячик, материал: резина, вид животного: собаки...",3,2,605,835,811,4.5,4
5,250197068,Игрушка для собак KONG Extreme L (K1),"игрушка, материал: резина, вид животного: соба...",5,0,835,1136,871,5,20
6,250199501,Игрушка для кошек Petstages Dental Мятный лист...,"игрушка, материал: текстиль, вид животного: ко...",9,15,152,271,169,4,34
7,250199624,Игрушка для кошек Petstages Play Трек 3 этажа ...,"игрушка, материал: пластик, вид животного: кошки",11,26,1059,1841,1184,4.5,89
8,251165051,Игрушка для собак Petstages Finity Dental Chew...,"игрушка, материал: пластик, резина, вид животн...",1,0,613,613,613,5,3
9,263734179,Игрушка для кошек Georplast Tricky,"игрушка, материал: пластик, вид животного: кошки",9,8,515,822,626,4,29


In [129]:
df[df.childCount == 0].head(20)

,id,name,fullName,link,level,parent,childCount
0,17392030,Упаковочные материалы для Беру,Упаковочные материалы для Беру,https://market.yandex.ru/catalog--upakovochnye...,0,NaN,0
16,15807107,Услуги по подписке,Услуги по подписке,https://market.yandex.ru/catalog--uslugi-po-po...,0,NaN,0
65,14312405,Гадания и предсказания,Товары для гаданий и предсказаний,https://market.yandex.ru/catalog--tovary-dlia-...,1,90801,0
68,2212983,Иконы,Иконы,https://market.yandex.ru/catalog--ikony/221298...,1,90801,0
70,91302,Подарочные наборы,Подарочные наборы,https://market.yandex.ru/catalog--podarochnye-...,1,90801,0
71,17728967,Видеофильмы_new,Фильмы и видео,https://market.yandex.ru/catalog--filmy-i-vide...,1,90801,0
72,91441,Туры,Туры,https://market.yandex.ru/catalog--tury/91441/l...,1,90801,0
74,280637,Флаги и гербы,Флаги и гербы,https://market.yandex.ru/catalog--flagi-i-gerb...,1,90801,0
76,17692696,Траурная флористика,Траурная флористика,https://market.yandex.ru/catalog--traurnaia-fl...,1,90801,0
77,17725598,Музыка_new,Музыкальные диски и пластинки,https://market.yandex.ru/catalog--muzykalnye-d...,1,90801,0


---

# Получение категории по EAN-13

In [1]:
def getMarketCategoryByEAN(ean):
    _uuid = uuid.uuid4().hex
    _baseUrl = 'https://mobile.market.yandex.net/market/white/v2.1.5/search?'
    _params = {
        '-3': '0',
        'pp': '507',
        'page': '1',
        'count': '15',
        'prefer_models': '1',
        'local_offers_first': '0',
        'rearr_factors': 'market_disable_parametric_search_for_white_except_parametric_specification=0;market_promo_by_user_cart_split=additional_categories_precise_references_191227',
        'fields': 'FILTERS,SORTS,FOUND_CATEGORIES,MODEL_CATEGORY,MODEL_DEFAULT_OFFER,MODEL_DISCOUNTS,MODEL_MEDIA,MODEL_OFFERS,MODEL_PHOTO,MODEL_PHOTOS,MODEL_PRICE,MODEL_RATING,MODEL_SPECIFICATION,OFFER_DISCOUNT,OFFER_PHOTO,OFFER_SHOP,SHOP_RATING,OFFER_ACTIVE_FILTERS,MODEL_FILTER_COLOR',
        #
        '-8': ean,
        'uuid': _uuid,
    }
    _headers = {
        'Accept': '*/*',
        'X-Platform': 'IOS',
        'Api-Platform': 'IOS',
        'Accept-Language': 'en-us',
        'Connection': 'keep-alive',
        'X-App-Version': '600.13.1',
        'X-Device-Type': 'SMARTPHONE',
        'Host': 'mobile.market.yandex.net',
        'Accept-Encoding': 'gzip, deflate, br',
        'User-Agent': 'WhiteMarket/600.13.1 (ru.yandex.ymarket; build:824; iOS 13.3.0; iPhone)'
    }
    
    _request1 = rq.get(
        url=_baseUrl,
        params=_params,
        headers=_headers
    )
    
    _categories = set()
    
    for item in _request1.json()['items']:
        try:
            _categories.add((i['category']['id'], i['category']['fullName']))
        except Exception:
            pass
    
    return list(_categories)

In [2]:
getMarketCategoryByEAN(4605817132102)

[]

In [3]:
! wget https://raw.githubusercontent.com/mgcrp/hse_chatbot_2020/master/data/categories.csv

--2020-05-25 10:29:57--  https://raw.githubusercontent.com/mgcrp/hse_chatbot_2020/master/data/categories.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.244.133
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 139636 (136K) [text/plain]
Сохранение в: «categories.csv»

categories.csv      100%[===================>] 136,36K   568KB/s    за 0,2s    

2020-05-25 10:29:58 (568 KB/s) - «categories.csv» сохранён [139636/139636]



In [4]:
df = pd.read_csv('categories.csv')
df.head(10)

,BU_Level1,BU_Level2,BU_Level3,BU_Level4
0,"Рукоделие, творчество, хобби",Хобби детское,Моделирование,Модель для сборки
1,"Рукоделие, творчество, хобби",Хобби детское,Моделирование,"Аксессуары, принадлежности для сборных моделей"
2,"Детские товары, игрушки",Товары для детской комнаты,Декор для детской,Часы для детской комнаты
3,Канцелярские товары,Чертежные принадлежности,"Линейка, транспортир",Набор геометрический
4,"Парфюмерия, косметика",Парфюмерия,Парфюмерия для мужчин,Туалетная вода мужская
5,"Парфюмерия, косметика",Косметика для ухода за кожей,Средства для бритья мужские,Средство после бритья мужское
6,"Парфюмерия, косметика",Косметика для ухода за кожей,Средства для бритья мужские,Средство для бритья мужское
7,Спортивные товары и товары для активного отдыха,Туризм,"Приготовление пищи, посуда туристическая",Фляга для туризма
8,Галантерея,Бижутерия,Брелок бижу,Бижу брелок
9,Спортивные товары и товары для активного отдыха,Туризм,"Приготовление пищи, посуда туристическая","Посуда для туризма, пикника"


---

# Получение категории по поисковому запросу

In [5]:
def getCategoryBySearchRequest(req):
    _uuid = uuid.uuid4().hex
    _baseUrl = 'https://ipa.touch.market.yandex.ru/api/v2'
    _headers = {
        "Host" : "ipa.touch.market.yandex.ru",
        "X-Test-Id" : "",
        "Connection" : "keep-alive",
        "Accept" : "*/*",
        "Accept-Language" : "en-us",
        "Accept-Encoding" : "gzip, deflate, br",
        "Content-Type" : "application/json",
        "Api-Platform" : "IOS",
        "User-Agent" : "WhiteMarket/600.19.1 (ru.yandex.ymarket; build:1172; iOS 13.3.1; iPhone)",
        "X-Platform" : "IOS",
        "Content-Length" : "314",
        "X-App-Version" : "600.19.1",
        "X-Device-Type" : "SMARTPHONE"
    }
    _params = {
        'name': 'resolveSearch',
        'rearr_factors': 'market_disable_parametric_search_for_white_except_parametric_specification=0',
        'sections': 'MEDICINE',
        'uuid': _uuid,
    }
    _payload = {
        "params": [
            {
                "filters": {
                    "allow-collapsing": "1",
                    "glfilter": [],
                    "grhow": "shop",
                    "onstock": "1",
                    "show-cutprice": "0"
                },
                "local-offers-first": 0,
                "numdoc": 15,
                "page": 1,
                "pp": "SEARCH",
                "rearr-factors": "search_offline_offers=1",
                "show-shops": "all",
                "show-vendors": "all",
                "text": str(req),
                "use-default-offers": "1"
            }
        ]
    }
    
    _request = rq.post(
        url=_baseUrl,
        headers=_headers,
        params=_params,
        data=json.dumps(_payload)
    )
    
    _maxCnt = -1
    _result = []
    for cat in _request.json()['collections']['intent']:
        if cat['ownCount'] > _maxCnt:
            _maxCnt = cat['ownCount']
            _result = [cat]
        elif cat['ownCount'] == _maxCnt:
            _result.append(cat)
            
    return _result

In [6]:
getCategoryBySearchRequest('Часы для детской комнаты')

[{'entity': 'intent',
  'id': '15587458_71745',
  'navnodeId': 71745,
  'categoryId': 15587458,
  'name': 'Часы настенные',
  'uniqName': 'Настенные часы',
  'slug': 'chasy-nastennye',
  'isLeaf': True,
  'defaultOrder': 3,
  'ownCount': 148}]

In [ ]:
for index, row in df.iterrows():
    tmp = getCategoryBySearchRequest(row.BU_Level4)
    if len(tmp) == 1:
        df.at[index, 'yandex_category_id'] = str(tmp[0]['categoryId'])
        df.at[index, 'yandex_category_name'] = str(tmp[0]['name'])
    else:
        df.at[index, 'yandex_category_id'] = ','.join([str(i['categoryId']) for i in tmp])
        df.at[index, 'yandex_category_name'] = ','.join([str(i['name']) for i in tmp])

df.head(10)

In [ ]:
df.to_csv('categories_with_yandex.csv')

In [ ]:
df[df.yandex_category_id.apply(lambda x: x.count(',') > 0)]

---

# Получение информации о модели по ID

https://yandex.ru/dev/market/content-data/doc/dg-v2/reference/models-controller-v2-get-model-docpage/

In [8]:
def getDataByYandexID(yandex_id):
    output = {}
    
    # Part 1 - get basic information, such as name and description
    
    _uuid = uuid.uuid4().hex
    _baseUrl = f'https://mobile.market.yandex.net/market/white/v2.1.5/models/{yandex_id}'
    _params = {
        'count':'10',
        'fields':'SHOP_RATING,OFFER_OFFERS_LINK,OFFER_SHOP,OFFER_DELIVERY,OFFER_DISCOUNT,FILTERS,FILTER_FOUND,FILTER_PHOTO_PICKER,FILTER_SORTS,OFFER_ACTIVE_FILTERS,PHOTO',
        'groupBy':'SHOP',
        'local_offers_first':'0',
        'onstock':'1',
        'page':'1',
        'pp':'531',
        'sort':'RELEVANCY',
        'rearr_factors':'market_disable_parametric_search_for_white_except_parametric_specification=0',
        'sections':'MEDICINE',
        'uuid':_uuid
    }
    _headers = {
        'Host':	'mobile.market.yandex.net',
        'Accept':	'*/*',
        'Accept-Language':	'en-us',
        'Accept-Encoding':	'gzip, deflate, br',
        'X-Test-Id':	'',
        'Api-Platform':	'IOS',
        'User-Agent':	'WhiteMarket/600.19.1 (ru.yandex.ymarket; build:1172; iOS 13.3.1; iPhone)',
        'X-Platform':	'IOS',
        'X-App-Version':	'600.19.1',
        'Connection':	'keep-alive',
        'X-Device-Type':	'SMARTPHONE'
    }
    
    _request = rq.get(
        url=_baseUrl,
        headers=_headers,
        params=_params
    )
    
    output['name'] = _request.json()['model']['name']
    output['description'] = _request.json()['model']['description']
    output['market_link'] = _request.json()['model']['link']
    output['photo'] = _request.json()['model']['photo']['url']
    
    # Part 2 - get offers
    
    _params = {
        'count':'10',
        'fields':'SHOP_RATING,OFFER_OFFERS_LINK,OFFER_SHOP,OFFER_DELIVERY,OFFER_DISCOUNT,FILTERS,FILTER_FOUND,FILTER_PHOTO_PICKER,FILTER_SORTS,OFFER_ACTIVE_FILTERS',
        'groupBy':'SHOP',
        'local_offers_first':'0',
        'onstock':'1',
        'page':'1',
        'pp':'531',
        'sort':'RELEVANCY',
        'rearr_factors':'market_disable_parametric_search_for_white_except_parametric_specification=0',
        'sections':'MEDICINE',
        'uuid':_uuid
    }
    
    _offers = []
    _baseUrl = f'https://mobile.market.yandex.net/market/white/v2.1.5/models/{yandex_id}/offers'
    
    _response = rq.get(
        url=_baseUrl,
        headers=_headers,
        params=_params
    )
    
    _offers.extend(_response.json()['offers'])
    _curPage = 2
    _maxPage = _response.json()['context']['page']['total']
    
    while _curPage < _maxPage:
        _params['page'] = _curPage
        print(_params)
        _response = rq.get(
            url=_baseUrl,
            headers=_headers,
            params=_params
        )
        _offers.extend(_response.json()['offers'])
        _curPage += 1
        
    _prices = [float(offer['price']['value']) for offer in _offers]
    
    output['max_price'] = max(_prices)
    output['min_price'] = min(_prices)
    output['count_offers'] = len(_offers)
    output['best_offer'] = _offers[np.argmin(_prices)]
    
    # Part 3 - get specifications
    
    _baseUrl = f'https://mobile.market.yandex.net/market/white/v2.1.5/models/{yandex_id}/specification'
    _params = {
        'rearr_factors': 'market_disable_parametric_search_for_white_except_parametric_specification=0',
        'sections': 'MEDICINE',
        'uuid': _uuid
    }
    
    _response = rq.get(
        url=_baseUrl,
        headers=_headers,
        params=_params
    )
    output['specification'] = _response.json()['specification'][0]
    
    return output

In [9]:
getDataByYandexID(558168099)

{'count': '10', 'fields': 'SHOP_RATING,OFFER_OFFERS_LINK,OFFER_SHOP,OFFER_DELIVERY,OFFER_DISCOUNT,FILTERS,FILTER_FOUND,FILTER_PHOTO_PICKER,FILTER_SORTS,OFFER_ACTIVE_FILTERS', 'groupBy': 'SHOP', 'local_offers_first': '0', 'onstock': '1', 'page': 2, 'pp': '531', 'sort': 'RELEVANCY', 'rearr_factors': 'market_disable_parametric_search_for_white_except_parametric_specification=0', 'sections': 'MEDICINE', 'uuid': '19d12e737b9c4ec0ac7ecc23cad98309'}
{'count': '10', 'fields': 'SHOP_RATING,OFFER_OFFERS_LINK,OFFER_SHOP,OFFER_DELIVERY,OFFER_DISCOUNT,FILTERS,FILTER_FOUND,FILTER_PHOTO_PICKER,FILTER_SORTS,OFFER_ACTIVE_FILTERS', 'groupBy': 'SHOP', 'local_offers_first': '0', 'onstock': '1', 'page': 3, 'pp': '531', 'sort': 'RELEVANCY', 'rearr_factors': 'market_disable_parametric_search_for_white_except_parametric_specification=0', 'sections': 'MEDICINE', 'uuid': '19d12e737b9c4ec0ac7ecc23cad98309'}
{'count': '10', 'fields': 'SHOP_RATING,OFFER_OFFERS_LINK,OFFER_SHOP,OFFER_DELIVERY,OFFER_DISCOUNT,FILTERS,

{'name': 'Смартфон Apple iPhone 11 Pro Max 256GB',
 'description': 'тип: смартфон, линейка: iPhone 11, диагональ экрана: 6" и больше, разрешение экрана: 2688x1242, 4G LTE, 2 SIM-карты, память: 256 ГБ, NFC, количество основных камер: 3, год анонсирования: 2019',
 'market_link': 'https://market.yandex.ru/product--smartfon-apple-iphone-11-pro-max-256gb/558168099?pp=531&hid=91491',
 'photo': 'https://avatars.mds.yandex.net/get-mpic/1669769/img_id9092225037069666521.jpeg/orig',
 'max_price': 113990.0,
 'min_price': 96254.0,
 'count_offers': 60,
 'best_offer': {'name': 'Смартфон Apple iPhone 11 Pro Max (256GB,Space Gray)',
  'description': 'Гарагтия производителя. Смартфон с iOS 13 поддержка двух SIM-карт (nano SIM+eSIM) экран 6.5", разрешение 2688x1242 три камеры 12 МП/12 МП/12 МП, автофокус память 256 Гб, без слота для карт памяти 3G, 4G LTE, LTE-A, Wi-Fi, Bluetooth, NFC, GPS, ГЛОНАСС вес 226 г, ШxВxТ 77.80x158x8.10 мм',
  'price': {'value': '96254'},
  'url': 'https://market-click2.yandex

In [10]:
_uuid = uuid.uuid4().hex
_baseUrl = f'https://mobile.market.yandex.net/market/white/v2.1.5/offers/16336768'
_params = {
        'count':'10',
        'fields':'SHOP_RATING,OFFERS_LINK,SHOP,DELIVERY,DISCOUNT,PHOTO',
        'groupBy':'SHOP',
        'local_offers_first':'0',
        'onstock':'1',
        'page':'1',
        'pp':'531',
        'sort':'RELEVANCY',
        'rearr_factors':'market_disable_parametric_search_for_white_except_parametric_specification=0',
        'sections':'MEDICINE',
        'uuid':_uuid
    }
_headers = {
        'Host':	'mobile.market.yandex.net',
        'Accept':	'*/*',
        'Accept-Language':	'en-us',
        'Accept-Encoding':	'gzip, deflate, br',
        'X-Test-Id':	'',
        'Api-Platform':	'IOS',
        'User-Agent':	'WhiteMarket/600.19.1 (ru.yandex.ymarket; build:1172; iOS 13.3.1; iPhone)',
        'X-Platform':	'IOS',
        'X-App-Version':	'600.19.1',
        'Connection':	'keep-alive',
        'X-Device-Type':	'SMARTPHONE'
    }

    
_response = rq.get(
        url=_baseUrl,
        headers=_headers,
        params=_params
    )
_response.json()

{'status': 'ERROR',
 'context': {'region': {'id': 213,
   'name': 'Москва',
   'country': {'id': 225,
    'name': 'Россия',
    'type': 'COUNTRY',
    'childCount': 10},
   'type': 'CITY',
   'childCount': 14},
  'currency': {'id': 'RUR', 'name': 'руб.'},
  'id': '1590392353590/7e424030408a1a28db0b4a1374a60500',
  'time': '2020-05-25T10:39:13.653+03:00'},
 'errors': [{'message': 'Offer 16336768 not found'}]}

In [11]:
_baseUrl = 'https://mobile.market.yandex.net/market/white/v2.1.5/models/558168099/specification'
_params = {
    'rearr_factors': 'market_disable_parametric_search_for_white_except_parametric_specification=0',
    'sections': 'MEDICINE',
    'uuid': _uuid
}
_headers = {
        'Host':	'mobile.market.yandex.net',
        'Accept':	'*/*',
        'Accept-Language':	'en-us',
        'Accept-Encoding':	'gzip, deflate, br',
        'X-Test-Id':	'',
        'Api-Platform':	'IOS',
        'X-YANDEXUID':	'9613489451589470333',
        'User-Agent':	'WhiteMarket/600.19.1 (ru.yandex.ymarket; build:1172; iOS 13.3.1; iPhone)',
        'X-User-Authorization':	'OAuth: AgAAAAA9mGxuAAGvFxQtfruNv05VpKEXPoiDkl8',
        'X-Platform':	'IOS',
        'X-App-Version':	'600.19.1',
        'Connection':	'keep-alive',
        'Cookie':	'yandexuid=9613489451589470333',
        'X-Device-Type':	'SMARTPHONE'
    }

_response = rq.get(
        url=_baseUrl,
        headers=_headers,
        params=_params
    )
_response.json()

{'context': {'region': {'id': 213,
   'name': 'Москва',
   'country': {'id': 225,
    'name': 'Россия',
    'type': 'COUNTRY',
    'childCount': 10},
   'type': 'CITY',
   'childCount': 14},
  'currency': {'id': 'RUR', 'name': 'руб.'},
  'id': '1590392354634/24244cb3f8c20b7c5afb591374a60500',
  'time': '2020-05-25T10:39:14.886+03:00'},
 'status': 'OK',
 'link': 'https://market.yandex.ru/product--smartfon-apple-iphone-11-pro-max-256gb/558168099/spec?pp=37&hid=91491',
 'specification': [{'name': 'Общие характеристики',
   'features': [{'name': 'Тип', 'value': 'смартфон'},
    {'name': 'Версия ОС на начало продаж', 'value': 'iOS 13'},
    {'name': 'Тип корпуса', 'value': 'классический'},
    {'name': 'Конструкция', 'value': 'влагозащита'},
    {'name': 'Количество SIM-карт', 'value': '2'},
    {'name': 'Тип SIM-карты', 'value': 'nano SIM+eSIM'},
    {'name': 'Бесконтактная оплата', 'value': 'есть'},
    {'name': 'Вес', 'value': '226 г'},
    {'name': 'Размеры (ШxВxТ)', 'value': '77.8x158x